In [1]:
# 구글 드라이브 마운트
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!ls ./drive/MyDrive/Colab\ Notebooks
#!ls /content/notebooks
# !rm -r /content/notebooks

# !ls /content/drive/MyDrive/Google\ Colaboratory

# working directory
#!ls ./drive/MyDrive/
%cd /content/drive/MyDrive/Google\ Colaboratory/FeatureSelectionAnalysis2  
!pwd


/content/drive/MyDrive/Google Colaboratory/FeatureSelectionAnalysis2
/content/drive/MyDrive/Google Colaboratory/FeatureSelectionAnalysis2


In [3]:
# 라이브러리 저장할 환경 path 설정

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
# if os.path.isdir(r'/content/drive/MyDrive/Google Colaboratory/EnsembleLogisticRegression/my_env'):
# print("my_env is found")
os.symlink(r'/content/drive/MyDrive/Google Colaboratory/FeatureSelectionAnalysis2/my_env', my_path)
sys.path.insert(0, my_path)

In [ ]:
my_path = '/content/notebooks'
sys.path.insert(0, my_path)

In [ ]:
# 라이브러리 설치
# !pip install --target=$my_path numpy==1.18.1
# !pip install --target=$my_path scipy==1.1.0
# !pip install --target=$my_path scikit-image==0.17.2
# !pip install --target=$my_path scikit-learn==0.22.1
# !pip install --target=$my_path nibabel==3.0.2

# !pip install --target=$my_path pydicom==1.4.2
# !pip install --target=$my_path keras==2.2.4
# !pip install --target=$my_path keras-applications==1.0.8

# !pip install --target=$my_path tensorflow-gpu==1.10.0
!pip install --target=$my_path pycasso
#!pip list


     |████████████████████████████████| 2.8MB 13.3MB/s 
     |████████████████████████████████| 15.3MB 259kB/s 
     |████████████████████████████████| 27.4MB 1.2MB/s 
  Created wheel for pycasso: filename=pycasso-0.1.2.2-cp37-none-any.whl size=3625643 sha256=f2748fb163f47ade9b1a31a16f3ba7becfdc708379f26f1c6d438d51a1c539e1
  Stored in directory: /root/.cache/pip/wheels/f4/f5/c1/fe83a055ef85cad69d90852324bcc20fd7df044d01ae24c819
Successfully built pycasso
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [5]:
# import library
import os 
import random
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import f_oneway
from statsmodels.stats import multitest
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

import pycasso



/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
def f_test_for_df(input_df, dep_colname):
  """
  This function will allow f-test to evaluate the association between selected independent and target variables
  and make the pair of (independent name and the p value after f_test).

  :param input_df: dataframe as a tabular data such as (N, F) including independent and dependent variables 
  :param dep_colname: column name to pick dependent variables

  :return: (ind_colnames, res_pval), 
  """
  
  ind_colnames = []
  res_pval = []
  colnames = input_df.columns
  
  dep_elems = input_df[dep_colname]
  for colname in colnames:
    if colname==dep_colname:
      continue
    else:
      ind_elems = input_df[colname]
      res = f_oneway(dep_elems, ind_elems)
      res_pval.append(res[1])
      ind_colnames.append(colname)
  return ind_colnames, res_pval

In [10]:
# load data
data_dir = r'/content/drive/MyDrive/Google Colaboratory/FeatureSelectionAnalysis2/datas/tutorial_data.xlsx'
dat = pd.read_excel(data_dir)
print(dat)


   Sex  Age  Onset age  Education  ...  MMSE  CDR  CDR-SOB  GDS
0    0   72         68          6  ...    17  1.0      6.0    5
1    1   55         53         12  ...    16  1.0      8.0    4
2    1   76         67          0  ...    14  0.5      4.0    4
3    0   72         67         16  ...    19  1.0      7.0    5
4    0   60         55          6  ...    19  0.5      3.5    4

[5 rows x 12 columns]


In [17]:
# 목표 : 각 독립변수들과 하나의 종속 변수 간에 f-test를 적용한 p-value를 계산하고 오름차순으로 정렬

# ranking by f_test
dep_colname = 'BAPL score'
ind_colnames, res_pval = f_test_for_df(dat, dep_colname)
# print(len(ind_colnames), ind_colnames)
# print(len(res_pval), res_pval)

# sort res_pval
pval_sort_ind = np.argsort(res_pval)
print("Column names ranked by p-values", np.array(ind_colnames)[pval_sort_ind])
print("The p-values", np.array(res_pval)[pval_sort_ind])

Column names ranked by p-values ['Onset age' 'Age' 'MMSE' 'Diagnosis code_1' 'CDR-SOB' 'GDS' 'Sex' 'CDR'
 'Amyloid positivity (0,1)' 'PET ligand type' 'Education']
The p-values [9.97898504e-08 2.48156007e-07 1.15773912e-06 2.20525397e-03
 1.36719844e-02 1.49785443e-02 1.49785443e-02 3.09957088e-02
 3.52652035e-02 4.79117261e-02 8.24253914e-02]


In [18]:
# 목표 : 선택된 종속변수와의 연관성에 따라 정렬된 독립변수를 누적하면서 예측 모델을 만들고 RMSE 측정   

# experimental parameters
seed = 1
method = 'mcp'# 'l1' # 'mcp' # 'scad'
family = 'gaussian' 
"""Options for model. Sparse linear regression and sparse multivariate regression is applied if
            `family = "gaussian"`, sqrt lasso is applied if `family = "sqrtlasso"`, sparse logistic regression is
            applied if `family = "binomial"` and sparse poisson regression is applied if `family = "poisson"`.
            The default value is `"gaussian"`."""

# data split
X_train_df, X_test_df = train_test_split(dat, test_size = 0.3, random_state=seed) # 70% training data 생성
print("X_train_df", X_train_df.shape)
print("X_test_df", X_test_df.shape)

# experiment
rmse_list = []
mae_list = []
dep_colname_list = []
for ind in pval_sort_ind:
  
  dep_colname_list.append(ind_colnames[ind])
  print("dep_colname_list", dep_colname_list)

  X = X_train_df[dep_colname_list].to_numpy()
  y = X_train_df[dep_colname].to_numpy()
  solver = pycasso.Solver(X, y, family=family, penalty=method)
  solver.train()

  y_ = solver.predict(X_test_df[dep_colname_list].to_numpy())
  print("y_", y_.shape)
  rmse = mean_squared_error(X_test_df[dep_colname].to_numpy(), y_)
  mae = mean_absolute_error(X_test_df[dep_colname].to_numpy(), y_)
  rmse_list.append(rmse)
  mae_list.append(mae)
  del solver

print("rmse_list", rmse_list)
print("mae_list", mae_list)

X_train_df (3, 12)
X_test_df (2, 12)
dep_colname_list ['Onset age']
Training is over.
y_ (2,)
dep_colname_list ['Onset age', 'Age']
Training is over.
y_ (2,)
dep_colname_list ['Onset age', 'Age', 'MMSE']
Training is over.
y_ (2,)
dep_colname_list ['Onset age', 'Age', 'MMSE', 'Diagnosis code_1']
Training is over.
y_ (2,)
dep_colname_list ['Onset age', 'Age', 'MMSE', 'Diagnosis code_1', 'CDR-SOB']
Training is over.
y_ (2,)
dep_colname_list ['Onset age', 'Age', 'MMSE', 'Diagnosis code_1', 'CDR-SOB', 'GDS']
Training is over.
y_ (2,)
dep_colname_list ['Onset age', 'Age', 'MMSE', 'Diagnosis code_1', 'CDR-SOB', 'GDS', 'Sex']
Training is over.
y_ (2,)
dep_colname_list ['Onset age', 'Age', 'MMSE', 'Diagnosis code_1', 'CDR-SOB', 'GDS', 'Sex', 'CDR']
Training is over.
y_ (2,)
dep_colname_list ['Onset age', 'Age', 'MMSE', 'Diagnosis code_1', 'CDR-SOB', 'GDS', 'Sex', 'CDR', 'Amyloid positivity (0,1)']
Training is over.
y_ (2,)
dep_colname_list ['Onset age', 'Age', 'MMSE', 'Diagnosis code_1', 'CDR-S